In [37]:
# 统计 pitch / duration分布直方图的overlap
import io
import os
import shutil

# output_dir = 'l2m'

In [38]:
SEP = '[sep]'
ALIGN = '[align]'
beam = 5

In [39]:
input_lyric_file = '/home/dwj/data/code/model_modify/zhonghao/songmass/data/test.lyric'
ground_truth_melody_file = '/home/dwj/data/code/model_modify/zhonghao/songmass/data/test.melody'
generate_melody_file = '/home/dwj/data/code/model_modify/zhonghao/songmass/generate/2021_7_7_test.log'
song_id_file = '/home/dwj/data/code/model_modify/zhonghao/songmass/data/test_song_id.txt'
generated_file = True
get_last = True
find_structure = True
cut_exceed_sent = True

In [40]:
def get_pitch_duration_structure(note_seq):
    seq = []
    
    i = 0
    while (i<len(note_seq)):
        if note_seq[i] > 128:
            #Duration
            i += 1
            continue
        else:
            #Pitch
            if i+1>=len(note_seq):
                #No Duration Followed
                break
            if note_seq[i+1] <= 128:
                #Followed by a pitch
                i += 1
                continue
            
            #Followed by a duration
#             pitch = note_seq[i]
#             duration = float(Duration_vocab[note_seq[i+1]])
            
            #Here trans back to str for bleu calculate
            pitch = str(note_seq[i])
            duration = str(note_seq[i+1])
            
            seq.append(pitch)
            seq.append(duration)
            i += 2
    return seq

In [41]:
def separate_sentences(x,find_structure = False):
    lst = x.copy()
    sep_positions = [i for i,x in enumerate(lst) if x==SEP]
    sep_positions.insert(0,-1)

    ret = []
    for i in range(len(sep_positions)-1):
        sent = lst[sep_positions[i]+1:sep_positions[i+1]] #SZH: not include sep token
        if find_structure:
            sent = list(map(int, sent))
            sent = get_pitch_duration_structure(sent)
        ret.append(sent)
    return ret

In [42]:
def get_songs(file, generated_file = False, get_last = False, find_structure = False, cut_exceed_sent=False):
    """
    Get Last : Whether include the last tokens if the sequence not ends with a seperation token
    """
    with io.open(input_lyric_file,'r') as f:
        input_lyrics= f.readlines()
    input_lyrics = list(map(lambda x:x.rstrip('\n').split(' '), input_lyrics))
    input_lyrics_sent_num = list(map(lambda x:x.count(SEP), input_lyrics))
    
    with io.open(song_id_file,'r') as f:
        song_ids = f.readlines()
    song_ids = list(map(lambda x:int(x.rstrip('\n')),song_ids))   
    
    with io.open(file,'r') as f:
        melody_lines = f.readlines()
        if generated_file:
            melody_lines = list(filter(lambda x:x.startswith('H-'), melody_lines))
            if len(melody_lines) == len(input_lyrics) * beam:
                melody_lines.sort( key=lambda x:(int(x.split('\t')[0].split('-')[1]), -float(x.split('\t')[1]) ) )
                melody_lines = [ x for i,x in enumerate(melody_lines) if i%beam == 0 ]
            else:    
                melody_lines.sort( key=lambda x:int(x.split('\t')[0].split('-')[1]) )
    melody_lines = list(map(lambda x:x.rstrip('\n').split('\t')[-1] ,melody_lines))
            
    print(len(melody_lines), len(input_lyrics))
    assert len(melody_lines)==len(input_lyrics)
    
    melody_seqs = list(map(lambda x:x.rstrip('\n').split(' '),melody_lines))
    melody_seqs = [ch for ch in melody_seqs if ch != ALIGN]
    for i in range(len(melody_seqs)):
        melody_seqs[i] = list(filter(lambda x:x.isdigit() or x==SEP, melody_seqs[i]))
        
    if get_last:
        for i in range(len(melody_seqs)):
            if melody_seqs[i][-1] != SEP:
                melody_seqs[i].append(SEP)
    melody_seq_sents = list(map(lambda x:separate_sentences(x,find_structure=find_structure), melody_seqs))
    return_list = []
    for i,sent_seq in enumerate(melody_seq_sents):
        if cut_exceed_sent and len(sent_seq) > input_lyrics_sent_num[i]:
            sent_seq = sent_seq[0:input_lyrics_sent_num[i]]
        cur_song_return = []

        for j,sent in enumerate(sent_seq):
            cur_song_return.extend(sent)
            cur_song_return.append(SEP)
        return_list.append(cur_song_return)
    song_num = song_ids[-1] + 1
    songs = [ [] for _ in range(song_num)]
    for item_id,song_id in enumerate(song_ids):
        songs[song_id].extend(return_list[item_id])
    
    return songs
            

In [43]:
ground_truth_melody = get_songs(ground_truth_melody_file)

714 714


In [44]:
output_melodies = get_songs(generate_melody_file, generated_file, get_last, find_structure, cut_exceed_sent)

714 714


In [45]:
def to_tuple(x):
    pitch_and_duration = [ch for ch in x if ch != SEP and ch != ALIGN]
    return [(pitch_and_duration[2*i], pitch_and_duration[2*i+1]) for i in range(len(pitch_and_duration)//2)]
ground_truth_melody = list(map(to_tuple, ground_truth_melody))
output_melodies = list(map(to_tuple, output_melodies))

In [46]:
import numpy as np

In [47]:
def get_pitch_count(x):
    cnt = [0] * 129
    for note in x:
        cnt[int(note[0])] += 1
    return np.array(cnt)
   
def get_duration_count(x):
    cnt = [ 0 for i,x in enumerate(list(range(25,3325,25)))]
    for note in x:
        cnt[int(note[1])-129] += 1
    return np.array(cnt)

In [48]:
# Pitch, 按歌曲, 算休止符
pitch_overlap = 0

for i in range(len(ground_truth_melody)):
    pitch_cnt_ground = get_pitch_count(ground_truth_melody[i]) 
    pitch_cnt_generate = get_pitch_count(output_melodies[i])
    sum_ground = np.sum(pitch_cnt_ground) if np.sum(pitch_cnt_ground) > 0 else 1
    sum_generate = np.sum(pitch_cnt_generate) if np.sum(pitch_cnt_generate) > 0 else 1
    
    pitch_dist_ground = pitch_cnt_ground.astype(np.float32) /sum_ground
    pitch_dist_generate = pitch_cnt_generate.astype(np.float32) /sum_generate
    
    pitch_diff = np.abs(pitch_dist_ground - pitch_dist_generate)
    pitch_overlap_sent = (pitch_dist_ground + pitch_dist_generate - pitch_diff)/2
    
    pitch_overlap += np.sum(pitch_overlap_sent)
    
print(pitch_overlap/len(ground_truth_melody))

0.511499088637353


In [50]:
# Duration 按歌曲, 算休止符
duration_overlap = 0

for i in range(len(ground_truth_melody)):
    duration_cnt_ground = get_duration_count(ground_truth_melody[i]) 
    duration_cnt_generate = get_duration_count(output_melodies[i])
    
    sum_ground = np.sum(duration_cnt_ground) if np.sum(duration_cnt_ground) > 0 else 1
    sum_generate = np.sum(duration_cnt_generate) if np.sum(duration_cnt_generate) > 0 else 1
    
    duration_dist_ground = duration_cnt_ground.astype(np.float32) /sum_ground
    duration_dist_generate = duration_cnt_generate.astype(np.float32) /sum_generate
    
    duration_diff = np.abs(duration_dist_ground - duration_dist_generate)
    duration_overlap_sent = (duration_dist_ground + duration_dist_generate - duration_diff)/2
    
    duration_overlap += np.sum(duration_overlap_sent)
    
print(duration_overlap/len(ground_truth_melody))

0.6371850577746709
